In [ ]:
import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
from sklearn.impute import SimpleImputer

# 데이터 확인

In [ ]:
train = pd.read_csv('../input/aicompetition/train.csv')
test = pd.read_csv('../input/aicompetition/test.csv')
submission = pd.read_csv('../input/aicompetition/sample_submission.csv')

In [ ]:
train

In [ ]:
test

In [ ]:
submission

In [ ]:
train.shape, test.shape, submission.shape

In [ ]:
train.columns

# 데이터 전처리

In [ ]:
#train_on = train['온도']
#test_on = test['온도']

### 결측치 처리1 - sklearn, missingpy...

- nan_euclidean_distances

In [ ]:
#from sklearn.metrics.pairwise import nan_euclidean_distances

#euclidean_train = nan_euclidean_distances(train,train)
#euclidean_test = nan_euclidean_distances(test,test)

- KNNImputer

In [ ]:
from sklearn.impute import KNNImputer

imputer = KNNImputer(n_neighbors = 15) #n_neighbors = 3/5/15/20 , weights="uniform"
train_imputed = imputer.fit_transform(train.drop(['label_역률평균', 'label_전류고조파평균', 'label_전압고조파평균'],axis=1))
test_imputed = imputer.fit_transform(test)
train_imputed = pd.DataFrame(train_imputed, columns = train.drop(['label_역률평균', 'label_전류고조파평균', 'label_전압고조파평균'],axis=1).columns)
test_imputed = pd.DataFrame(test_imputed, columns = test.columns)

In [ ]:
train_imputed

In [ ]:
test_imputed

In [ ]:
train_imputed.isnull().sum()

In [ ]:
test_imputed.isnull().sum()

In [ ]:
train = pd.concat([train_imputed,train[['label_역률평균', 'label_전류고조파평균', 'label_전압고조파평균']]],axis=1)
test = test_imputed

In [ ]:
train

- Imputation Using Deep Learning(Datawig)

In [ ]:
#import datawig

#real_train, pred_train = datawig.utils.random_split(train)
#real_test, pred_test = datawig.utils.random_split(test)
#imputer = datawig.SimpleImputer(input_columns = ['누적전력량', '유효전력평균', '무효전력평균', '주파수', '전류평균', '상전압평균', '선간전압평균',
#       '온도', 'R상유효전력', 'R상무효전력', 'R상전류', 'R상전압', 'R상선간전압', 'S상유효전력', 'S상무효전력',
#       'S상전류', 'S상전압', 'S상선간전압', 'T상유효전력', 'T상무효전력', 'T상전류', 'T상전압', 'T상선간전압'], output_column = ['label_역률평균', 'label_전류고조파평균', 'label_전압고조파평균'],output_path = 'imputer_model')
#imputer.fit(real_train, num_epochs = 100)
#imputer.fit(real_test, num_epochs = 100)
#imputed = imputer.predict(pred_train)
#imputed = imputer1.predict(pred_test)

- MissForest

In [ ]:
!pip install missingpy

In [ ]:
"""
from missingpy import MissForest

train_cols = train.columns.tolist()
test_cols = test.columns.tolist()

imputer = MissForest(random_state= 42)

train_imputed = imputer.fit_transform(train,cat_vars = train_cols)
test_imputed = imputer.fit_transform(test,cat_vars = test_cols)
train = pd.DataFrame(train_imputed, columns = train_cols)
test = pd.DataFrame(test_imputed, columns = test_cols)

#train_imputed = imputer.fit_transform(train.drop(['label_역률평균', 'label_전류고조파평균', 'label_전압고조파평균'],axis=1))
#test_imputed = imputer.fit_transform(test)
#train_imputed = pd.DataFrame(train_imputed, columns = train.drop(['label_역률평균', 'label_전류고조파평균', 'label_전압고조파평균'],axis=1).columns)
#test_imputed = pd.DataFrame(test_imputed, columns = test.columns)
"""

### 결측치 처리2 - 온도별 그룹 평균 처리

'온도' 컬럼 소수점 변환
- 사람의 체온을 잴때도 소수점 1자리까지만 재기 때문에 소수점 1자리로 사용
- ex) 35.6도

In [ ]:
#train['온도'] = round(train['온도'],1) # 소수점 1자리
#test['온도'] = round(test['온도'],1) # 소수점 1자리

In [ ]:
#train.columns

In [ ]:
"""
df_columns = {'누적전력량', '유효전력평균', '무효전력평균', '주파수', '전류평균', '상전압평균', '선간전압평균',
       '온도', 'R상유효전력', 'R상무효전력', 'R상전류', 'R상전압', 'R상선간전압', 'S상유효전력', 'S상무효전력',
       'S상전류', 'S상전압', 'S상선간전압', 'T상유효전력', 'T상무효전력', 'T상전류', 'T상전압', 'T상선간전압'}
       
train['온도'] = train['온도'].fillna(0)
test['온도'] = test['온도'].fillna(0)

#for col in df_columns:
#    if train[col][train['온도'] == 0.0].isnull:
#        train[col] = train[col].fillna(0)
#        test[col] = test[col].fillna(0)
        #train[col] = train[col].loc[train['온도'] == 0].fillna(0)
        #test[col] = test[col].loc[test['온도'] == 0].fillna(0)

#for col in df_columns:
#    train[col] = train[col].fillna(0)
#    test[col] = test[col].fillna(0)
        
        
for col in df_columns:
    train[col] = train[col].fillna(train.groupby(['온도'])[col].transform('mean'))
    test[col] = test[col].fillna(test.groupby(['온도'])[col].transform('mean'))

#train_not = train.drop(['label_역률평균','label_전류고조파평균','label_전압고조파평균'],axis=1)
#imputer = SimpleImputer(missing_values=np.nan, strategy='mean') # median, most_frequent
#train_not = imputer.fit_transform(train_not.values)
#test = imputer.fit_transform(test.values)
"""

In [ ]:
train.isnull().sum()

In [ ]:
test.isnull().sum()

기술통계
- 온도 별 기술통계
- 주파수 별 기술통계

In [ ]:
from scipy.stats import skew
from scipy.stats import kurtosis

def range_func(x):
    max_val = np.max(x)
    min_val = np.min(x)
    range_val = max_val - min_val
    return range_val

def iqr_func(x):
    low = 5
    high = 95
    q3, q1 = np.percentile(x, [low, high])
    iqr = q3 - q1
    return iqr

# mad(median_absolute_devation)
def premad(x):
    return np.median(np.absolute(x-np.median(x,axis=0)),axis=0)

# skewness, kurtosis
def preskew(x):
    return skew(x)

def prekurt(x):
    return kurtosis(x,fisher=True)


# 온도 별 기술통계
train_not_feature = train.columns[(train.columns.str.contains('평균') == False) & (train.columns.str.contains('label_') == False) & (train.columns.str.contains('index') == False) & (train.columns.str.contains('cos') == False)] # & (train.columns.str.contains('주파수') == False)
test_not_feature = test.columns[(test.columns.str.contains('평균') == False) & (test.columns.str.contains('ID') == False) & (test.columns.str.contains('cos') == False)] # & (test.columns.str.contains('주파수') == False)
train_preprocess = train[train_not_feature].groupby('주파수').agg(['min','max','mean','median']) #,premad,preskew,prekurt
test_preprocess = test[test_not_feature].groupby('주파수').agg(['min','max','mean','median'])
train_preprocess.columns=[i[0]+'_'+i[1] for i in train_preprocess.columns]
test_preprocess.columns=[i[0]+'_'+i[1] for i in test_preprocess.columns]

In [ ]:
train_preprocess

In [ ]:
test_preprocess

전력
- 전력(P) = 전압(V)*전류(I)

In [ ]:
train['R상전력'] = train['R상전압'] * train['R상전류']
test['R상전력'] = test['R상전압'] * test['R상전류']

train['S상전력'] = train['S상전압'] * train['S상전류']
test['S상전력'] = test['S상전압'] * test['S상전류']

train['T상전력'] = train['T상전압'] * train['T상전류']
test['T상전력'] = test['T상전압'] * test['T상전류']

피상전력
- 피상전력 = 유효전력^2 + 무효전력^2
- 유효전력 = 피상전력 * cos

In [ ]:
train['피상전력평균'] = (train['유효전력평균'].astype(float))**2 + (train['무효전력평균'].astype(float))**2
test['피상전력평균'] = (test['유효전력평균'].astype(float))**2 + (test['무효전력평균'].astype(float))**2

In [ ]:
train['R상피상전력'] = (train['R상유효전력'].astype(float))**2 + (train['R상무효전력'].astype(float))**2
test['R상피상전력'] = (test['R상유효전력'].astype(float))**2 + (test['R상무효전력'].astype(float))**2

train['S상피상전력'] = (train['S상유효전력'].astype(float))**2 + (train['S상무효전력'].astype(float))**2
test['S상피상전력'] = (test['S상유효전력'].astype(float))**2 + (test['S상무효전력'].astype(float))**2

train['T상피상전력'] = (train['T상유효전력'].astype(float))**2 + (train['T상무효전력'].astype(float))**2
test['T상피상전력'] = (test['T상유효전력'].astype(float))**2 + (test['T상무효전력'].astype(float))**2

역률 (power factor)
- 역률 cos = 유효전력/피상전력

In [ ]:
train['R상역률cos'] = train['R상유효전력']/train['R상피상전력']
test['R상역률cos'] = test['R상유효전력']/test['R상피상전력']

train['S상역률cos'] = train['S상유효전력']/train['S상피상전력']
test['S상역률cos'] = test['S상유효전력']/test['S상피상전력']

train['T상역률cos'] = train['T상유효전력']/train['T상피상전력']
test['T상역률cos'] = test['T상유효전력']/test['T상피상전력']

전류, 전압, 피상전력

In [ ]:
train['전류'] = train['R상전류'] + train['S상전류'] + train['T상전류']
train['전압'] = train['R상전압'] + train['S상전압'] + train['T상전압']
train['전력'] = train['R상전력'] + train['S상전력'] + train['T상전력']
train['피상전력'] = train['R상피상전력'] + train['S상피상전력'] + train['T상피상전력']

test['전류'] = test['R상전류'] + test['S상전류'] + test['T상전류']
test['전압'] = test['R상전압'] + test['S상전압'] + test['T상전압']
test['전력'] = test['R상전력'] + test['S상전력'] + test['T상전력']
test['피상전력'] = test['R상피상전력'] + test['S상피상전력'] + test['T상피상전력']

In [ ]:
#온도 별 기술통계2
#train_RST_feature = {'주파수','R상전력','S상전력','T상전력','R상피상전력','S상피상전력','T상피상전력','전력','전압','전력','피상전력'}
#test_RST_feature = {'주파수','R상전력','S상전력','T상전력','R상피상전력','S상피상전력','T상피상전력','전력','전압','전력','피상전력'}
#train_preprocess2 = train[train_RST_feature].groupby('주파수').agg(['min','max','mean','median']) #,premad,preskew,prekurt
#test_preprocess2 = test[test_RST_feature].groupby('주파수').agg(['min','max','mean','median'])
#train_preprocess2.columns=[i[0]+'_'+i[1] for i in train_preprocess2.columns]
#test_preprocess2.columns=[i[0]+'_'+i[1] for i in test_preprocess2.columns]

In [ ]:
train_preprocess2

In [ ]:
test_preprocess2

In [ ]:
train.isnull().sum()

In [ ]:
test.isnull().sum()

In [ ]:
train = pd.merge(train,train_preprocess,on='주파수',how='left')
test = pd.merge(test,test_preprocess,on='주파수',how='left')

#train = pd.merge(train,train_preprocess2,on='온도',how='left')
#test = pd.merge(test,test_preprocess2,on='온도',how='left')

In [ ]:
train

In [ ]:
test

{'누적전력량', '유효전력평균', '무효전력평균', '주파수', '전류평균', '상전압평균', '선간전압평균',
       '온도', 'R상유효전력', 'R상무효전력', 'R상전류', 'R상전압', 'R상선간전압', 'S상유효전력', 'S상무효전력',
       'S상전류', 'S상전압', 'S상선간전압', 'T상유효전력', 'T상무효전력', 'T상전류', 'T상전압', 'T상선간전압'}

In [ ]:
train.isnull().sum()

In [ ]:
test.isnull().sum()

In [ ]:
train = train.replace([np.inf, -np.inf],0)
test = test.replace([np.inf, -np.inf],0)
train = train.fillna(0)
test = test.fillna(0)

푸리에 변환
- 적합한 컬럼을 푸리에 변환

In [ ]:
def prepuriae(x):
  fft = np.fft.fft(x) / len(x)
  fft_abs = abs(fft)
  return fft_abs

def prebackpuriae(x):
  fft = np.fft.ifft(x)*len(x)
  fff_abs = abs(fft)
  fff_abs
  return fff_abs

In [ ]:
#train[train_not_feature] = prepuriae(train[train_not_feature])
#test[test_not_feature] = prepuriae(test[test_not_feature])

#train['주파수_puriae'] = prepuriae(train['주파수'])
#test['주파수_puriae'] = prepuriae(test['주파수'])

In [ ]:
train

In [ ]:
test

In [ ]:
train.columns

In [ ]:
test.columns

df_columns2 = {'누적전력량', '유효전력평균', '무효전력평균', '주파수', '전류평균', '상전압평균', '선간전압평균',
       '온도', 'R상유효전력', 'R상무효전력', 'R상전류', 'R상전압', 'R상선간전압', 'S상유효전력', 'S상무효전력',
       'S상전류', 'S상전압', 'S상선간전압', 'T상유효전력', 'T상무효전력', 'T상전류', 'T상전압', 'T상선간전압','R상전력', 'S상전력', 'T상전력',
       '피상전력', 'R상역률cos', 'S상역률cos', 'T상역률cos'}

In [ ]:
"""
from sklearn.cluster import KMeans
model = KMeans(n_clusters=5,random_state=31) # 5, 6
model.fit(train.drop(['label_역률평균','label_전류고조파평균','label_전압고조파평균'], axis=1))
train_predict = model.predict(train.drop(['label_역률평균','label_전류고조파평균','label_전압고조파평균'], axis=1))
train['cluster']=train_predict

test_predict = model.predict(test)
test['cluster']=test_predict
"""

In [ ]:
train

In [ ]:
test

In [ ]:
train.to_csv('preprocessing2_train.csv',index=False)
test.to_csv('preprocessing2_test.csv',index=False)

### get preprocessing file
- train
- test

In [ ]:
import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
from sklearn.impute import SimpleImputer
train = pd.read_csv('../input/preprocessing2/preprocessing2_train.csv')
test = pd.read_csv('../input/preprocessing2/preprocessing2_test.csv')

In [ ]:
pip install catboost

In [ ]:
from sklearn.model_selection import train_test_split
from catboost import CatBoostClassifier
from sklearn.metrics import log_loss
from sklearn.model_selection import StratifiedKFold
from tensorflow.keras.utils import to_categorical
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense
from tensorflow.keras import optimizers
from sklearn.neighbors import KNeighborsClassifier
import lightgbm as lgb
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import ExtraTreesClassifier
from sklearn.metrics import f1_score
from sklearn.ensemble import RandomForestClassifier
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import GridSearchCV
from sklearn.multiclass import OneVsRestClassifier
from sklearn.multiclass import OneVsOneClassifier
from lightgbm import LGBMClassifier

In [ ]:
train.rename(columns={'index':'ID'},inplace=True) 

In [ ]:
#train['온도'] = train_on
#test['온도'] = test_on

In [ ]:
train

In [ ]:
test

In [ ]:
train_x = train.drop(['label_역률평균','label_전류고조파평균','label_전압고조파평균'],axis=1)
train_y = train[['label_역률평균','label_전류고조파평균','label_전압고조파평균']]

In [ ]:
(X_train, X_test, y_train, y_test) = train_test_split(train_x, train_y, test_size=0.25,stratify=train_y) #0.25, 0.35

In [ ]:
# train test 비율 맞추기
print(X_test.shape[0]) # 605142
print(test.shape[0]) # 313267

In [ ]:
X_train.shape, y_train.shape,X_test.shape,y_test.shape

In [ ]:
from sklearn.preprocessing import StandardScaler,MinMaxScaler,MaxAbsScaler,Normalizer,QuantileTransformer,PowerTransformer # RobustScaler <- kaggle에서는 안됨

#caler = StandardScaler() -RAM error
#scaler = MinMaxScaler() - 3
#caler = MaxAbsScaler() -2
#caler = Normalizer() - 5
#scaler = QuantileTransformer() - 4
scaler = QuantileTransformer(output_distribution = 'normal') #,n_quantiles=100
#scaler = PowerTransformer() - 1


scaler.fit(X_train)
X_train = scaler.transform(X_train)
X_test = scaler.transform(X_test)
test = scaler.transform(test)

In [ ]:
# clf_params = {'min_samples_split':40,'max_leaf_nodes':15,'n_estimators':3000,'max_depth':7,'min_samples_leaf':3}
#clf_params = {'max_depth':6}

In [ ]:
from sklearn.model_selection import GridSearchCV
from sklearn.metrics import accuracy_score

 param_grid = {
     'n_estimators' : [10,30,50,100,150,200],
     'max_depth' : [None,6,8,10,12],
     'min_samples_leaf': [None,3,5,7,8,10,12,18],
     'min_sample_split' : [2,3,5,8,10,16,20],
     'max_features':[None,'sqrt','log2',5,6,7,8,9,10],
     'bootstrap':[True,False],
     'criterion':['gini','entropy']
        }
    
#param_grid = {'n_estimators':[10,1,2,3,4,5,6,7,8,9,100,200,300,1000], 'bootstrap':[True,False], 'criterion':['gini','entropy'], 'max_depth':[None,2,3,4,5,6], 'max_leaf_nodes':[None,2,3,4,5,6,7], 'min_samples_split':[2,3,4,5,6], 'min_samples_leaf':[1,2,3]}
#grid_search = GridSearchCV(clf1,param_grid=param_grid,cv=5,n_jobs=-1)
rf_clf = RandomForestClassifier(n_jobs = -1)
grid_search = GridSearchCV(rf_clf, param_grid=param_grid,cv=3) 
grid_search.fit(X_train,y_train)
print(grid_search.best_params_)
print(grid_search.best_score_)
# **grid.best_params_
df = pd.DataFrame(grid_search.cv_results_)
print(df)

In [ ]:
#knn = KNeighborsClassifier(n_neighbors=3)
#knn.fit(X_train,y_train)
clf1=RandomForestClassifier() #, class_weight = 'balanced' **clf_params n_jobs=-1,random_state = 100
clf1.fit(X_train,y_train)
#clf2 = ExtraTreesClassifier(n_estimators=1000, n_jobs=-1)
#clf2.fit(X_train,y_train)
#clf1 = LGBMClassifier()
#clf1 = oneVsOneClassifier(clf1)

In [ ]:
#y_pred = knn.predict(test)
#y_pred1 = clf.predict(test)
y_pred2 = clf1.predict(test)
#y_pred3 = clf3.predict(test)

In [ ]:
y_pred2_proba=clf1.predict_proba(test)

In [ ]:
y_pred2

In [ ]:
y_pred2_proba

In [ ]:
# f1_score(test,y_pred2,average='macro')

In [ ]:
#y_pred2_copy = y_pred2.copy()
#y_pred2_proba_copy = y_pred2_proba.copy()
#submission_copy = submission.copy()
#y_pred2_copy[np.where(y_pred2_proba_copy < 0.5)] = '경고'

In [ ]:
#y_pred2_copy[y_pred2_copy != y_pred2]

In [ ]:
#y_pred2_copy

In [ ]:
#submission_copy = y_pred2_copy
#submission_copy.to_csv('pre_rt_control_baseline_ttf4.csv',index=False)

In [ ]:
# print(f1_score(y_test, y_pred2, average='macro')) # 대회 평가 산식인 Macro F1 확인

In [ ]:
"""
import matplotlib.pyplot as plt
import seaborn as sns
%matplotlib inline

ftr_importances_values = clf1.feature_importances_
ftr_importances = pd.Series(ftr_importances_values, index = train_x.columns)
ftr_top20 = ftr_importances.sort_values(ascending=False)[:20]

plt.figure(figsize=(8,6))
plt.title('Top 20 Feature Importances')
sns.barplot(x=ftr_top20, y=ftr_top20.index)
plt.show()
"""

In [ ]:
submission = pd.read_csv('../input/aicompetition/sample_submission.csv')

In [ ]:
submission[['label_역률평균','label_전류고조파평균','label_전압고조파평균']] = y_pred2
#submission[['label_역률평균','label_전류고조파평균','label_전압고조파평균']] = y_pred1
#submission[['label_역률평균','label_전류고조파평균','label_전압고조파평균']] = y_pred2

In [ ]:
submission.to_csv('precsv_last0_preprocessing1_baseline.csv',index=False)

In [ ]:
submission

In [ ]:
submission['label_역률평균'].value_counts()

In [ ]:
#submission_copy['label_역률평균'].value_counts()

In [ ]:
submission['label_전류고조파평균'].value_counts()

In [ ]:
submission['label_전압고조파평균'].value_counts()

단일레이블 구현 예측

In [ ]:
def macro_f1(y_true, y_pred):
    score = f1_score(y_true=y_true, y_pred=y_pred, average='macro')
    return score

In [ ]:
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline
from sklearn.impute import SimpleImputer
from sklearn.preprocessing import StandardScaler, OneHotEncoder
from sklearn.experimental import enable_iterative_imputer
from sklearn.impute import IterativeImputer
from sklearn.metrics import confusion_matrix
from sklearn.metrics import roc_auc_score
from sklearn.metrics import classification_report
from sklearn.datasets import make_multilabel_classification
from xgboost import XGBClassifier
from sklearn.model_selection import KFold
from sklearn.multioutput import MultiOutputClassifier
from sklearn.pipeline import Pipeline
from lightgbm import LGBMClassifier

In [ ]:
from sklearn.model_selection import train_test_split
from sklearn.metrics import confusion_matrix
from sklearn.metrics import roc_auc_score
from sklearn.metrics import classification_report
from sklearn.datasets import make_multilabel_classification
from xgboost import XGBClassifier
from sklearn.model_selection import KFold
from sklearn.multioutput import MultiOutputClassifier
from sklearn.pipeline import Pipeline
import tensorflow as tf
from sklearn.preprocessing import LabelEncoder

In [ ]:
train_y = train[['label_역률평균','label_전류고조파평균','label_전압고조파평균']]
train_x = train.drop(['label_역률평균','label_전류고조파평균','label_전압고조파평균'],axis=1)
#train_x1 = train.drop('label_역률평균',axis=1)
#train_x2 = train.drop('label_전류고조파평균',axis=1)
#train_x3 = train.drop('label_전압고조파평균',axis=1)
train_y1 = train_y['label_역률평균']
train_y2 = train_y['label_전류고조파평균']
train_y3 = train_y['label_전압고조파평균']

In [ ]:
(X_train1, X_test1, y_train1, y_test1) = train_test_split(train_x, train_y1, test_size=0.3,stratify=train_y)
(X_train2, X_test2, y_train2, y_test2) = train_test_split(train_x, train_y2, test_size=0.3,stratify=train_y)
(X_train3, X_test3, y_train3, y_test3) = train_test_split(train_x, train_y3, test_size=0.3,stratify=train_y)

In [ ]:
#cat1 = CatBoostClassifier()
#cat2 = CatBoostClassifier()
#cat3 = CatBoostClassifier()
cat1 = LGBMClassifier()
cat2 = LGBMClassifier()
cat3 = LGBMClassifier()
cat1.fit(X_train1,y_train1)
cat2.fit(X_train2,y_train2)
cat3.fit(X_train3,y_train3)

In [ ]:
y_cat_pred1 = cat1.predict(test)
y_cat_pred2 = cat2.predict(test)
y_cat_pred3 = cat3.predict(test)

In [ ]:
y_cat_pred1

In [ ]:
y_cat_pred2

In [ ]:
y_cat_pred3

In [ ]:
submission['label_역률평균'] = y_cat_pred1
submission['label_전류고조파평균'] = y_cat_pred2
submission['label_전압고조파평균'] = y_cat_pred3

In [ ]:
submission['label_역률평균'].value_counts()

In [ ]:
submission['label_전류고조파평균'].value_counts()

In [ ]:
submission['label_전압고조파평균'].value_counts()

In [ ]:
submission.to_csv('pre_lgbm_baseline_ttf.csv',index=False)

In [ ]:
#from sklearn import metrics
#metrics.f1_score(y_test, pred, average='macro')

### MultilabelStratifiedKfold

In [ ]:
from iterstrat.ml_stratifiers import MultilabelStratifiedKFold

multikfold = MultilabelStratifiedKFold(n_splits = 5, shuffle = True, random_state = 100)

for train_index, test_index  in multikfold.split(train_x,train_y):
    print('Train:',train_index, 'Test:',test_index)
    X_train, X_test = train_x[train_index], train_x[test_index]
    y_train, y_test = train_y[train_index], train_y[test_index]
    